### 获取根路径

In [1]:
import sys
import os

# 获取当前工作目录
# Trytorch\test
current_dir = os.getcwd()

# 获取父目录
# Trytorch
project_dir = os.path.dirname(current_dir)

# 现在的路径即为项目根路径,可以找到主包trytorch
sys.path.append(project_dir)

In [2]:
import numpy as np

import trytorch as torch
import trytorch.ops as ops
import trytorch.nn as nn
import trytorch.optim as optim
import trytorch.datas as data
from trytorch.array_device import *

c:\Users\GameDesign\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


### 搭建网络,优化器,损失函数

In [3]:
batch_size = 100
epochs = 10
device = cpu()

class Model(nn.Module):
    def __init__(self, hidden = 100 , device = device):
        super().__init__()

        self.block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, hidden, device=device),
            nn.ReLU(),
            nn.Linear(hidden, 10, device=device),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.block(x)

net = Model()
optimizer = optim.Adam(net.parameters(),lr=0.001,weight_decay=0.001)
criterion = nn.SoftmaxLoss()

### dataset 与 dataloader 测试

In [4]:
from pathlib import Path

project_path = Path(project_dir)
print(project_path)

# 训练数据集
train_dataset = data.MNISTDataset(
    project_path / "data" / "MNIST" / "train-images-idx3-ubyte.gz",
    project_path / "data" / "MNIST" / "train-labels-idx1-ubyte.gz"
)

train_dataloader = data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_dataset = data.MNISTDataset(
    project_path / "data" / "MNIST" / "t10k-images-idx3-ubyte.gz",
    project_path / "data" / "MNIST" / "t10k-labels-idx1-ubyte.gz"
)

test_dataloader = data.DataLoader(
    dataset = test_dataset,
    batch_size=batch_size,
    shuffle=True
)


d:\AIExperienments\TryTorch


In [ ]:
import matplotlib.pyplot as plt

# 显示数据集

img, label = train_dataset[42]
# 因为是 (1, 28, 28)，需要 squeeze 去掉 channel 维度
plt.imshow(img.squeeze(), cmap="gray")
plt.title(f"Label: {label}")
plt.axis("off")
plt.show()

# 显示数据加载器
len(train_dataloader), len(test_dataloader)

### 🤗 训练

In [6]:
for epoch in range(epochs):
    total_loss = 0
    total_rights = 0
    total_examples = 0
    total_batches = 0
    for inputs, label in train_dataloader:
        
        net.train()
        
        optimizer.reset_grad()

        pred = net(inputs)

        loss = criterion(pred, label)

        loss.backward()

        optimizer.step()
        # (batch, features) -> (batch, 1)
        label_pred = np.argmax(pred.numpy(), axis = 1)

        rights = np.equal(label_pred, label.numpy()).sum()

        total_loss += loss.numpy()
        total_rights += rights
        total_batches += 1
        total_examples += inputs.shape[0]

    avg_loss = total_loss / total_batches
    avg_accuracy = total_rights / total_examples
    print(f"EPOCH {epoch}: {avg_accuracy=}, {avg_loss=}")



EPOCH 0: avg_accuracy=0.8038666666666666, avg_loss=0.6048535300890603
EPOCH 1: avg_accuracy=0.8598833333333333, avg_loss=0.40436179456710863
EPOCH 2: avg_accuracy=0.8702166666666666, avg_loss=0.36651684214274066
EPOCH 3: avg_accuracy=0.8745, avg_loss=0.3484018104712162
EPOCH 4: avg_accuracy=0.8773666666666666, avg_loss=0.3381053332487742
EPOCH 5: avg_accuracy=0.8794, avg_loss=0.33049678289095535
EPOCH 6: avg_accuracy=0.8810333333333333, avg_loss=0.3241536554018659
EPOCH 7: avg_accuracy=0.8818666666666667, avg_loss=0.3201875269412994
EPOCH 8: avg_accuracy=0.8827, avg_loss=0.3177771744410197
EPOCH 9: avg_accuracy=0.8830666666666667, avg_loss=0.31539576880137116


### 🤗 测试

In [7]:
total_loss = 0
total_rights = 0
total_examples = 0
total_batches = 0

with torch.no_grad():
    for inputs, label in test_dataloader:
        
        net.eval()

        pred = net(inputs)

        loss = criterion(pred, label)

        label_pred = np.argmax(pred.numpy(),axis=1)
        
        rights = np.equal(label_pred, label.numpy()).sum()

        total_loss += loss.numpy()
        total_rights += rights
        total_batches += 1
        total_examples += inputs.shape[0]

    avg_loss = total_loss / total_batches
    avg_accuracy = total_rights / total_examples
    print(f"TEST SCORE: {avg_accuracy=}, {avg_loss=}")

TEST SCORE: avg_accuracy=0.8791, avg_loss=0.32292039728164673
